In [1]:
import os
import numpy as np
import cv2

In [2]:
d=[]
idx2name={}
pics=[]
labels=[]
cnt=0
Y=[]
files=os.listdir()
for f in files:
    if f.endswith('.npy'):
        data=np.load(f)
        print('data shape',data.shape)
        labels=np.ones(data.shape[0],dtype='int32')*cnt
        pics.append(data)
        idx2name[cnt]=f[:-4]
        cnt+=1
        Y.append(labels)
        print("labels : ",labels)
        
idx2name

data shape (20, 100, 100, 3)
labels :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
data shape (20, 100, 100, 3)
labels :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
data shape (20, 100, 100, 3)
labels :  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
data shape (20, 100, 100, 3)
labels :  [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


{0: 'Aarna', 1: 'Karuna', 2: 'Sameer', 3: 'Sushantika'}

In [3]:
Y

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])]

In [4]:
def dist(a,b):
    return np.sum(((a-b)**2)**.5)

In [5]:
def knn(X,Y,test_pt,k=5):
    d=[]
    m=X[1].shape[0]
    #print('0 X Y test',X[0].shape,Y[0].shape,test_pt.shape)
    
    for i in range(len(pics)):
        cnt=0
        for j in range(m):
            cnt+=1
            #print('2',cnt,' X Y test',X[i][j].shape,Y[i].shape,test_pt.shape)
            c_dis=dist(X[i][j],test_pt)
            #print(c_dis)
            d.append((c_dis,Y[i][j]))
    d.sort()
    d=d[0:k]
    d=np.array(d)
    #print('d shape',d.shape)
    d=d[:,1]
    #print('d ',d)
    uniq,occ=np.unique(d,return_counts=True)
    idx=np.argmax(occ)
    pred=uniq[idx]
    
    return int(pred)

In [5]:
# xxx=[1,2,2,1,5,6,2]
# u,oc=np.unique(xxx,return_counts=True)

In [8]:
# print(u," , ",oc)

[1 2 5 6]  ,  [2 3 1 1]


In [3]:
face_detector=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [7]:
cam=cv2.VideoCapture(0)
while True:
    b,img=cam.read()
    #cv2.imshow('img',img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #cv2.imshow("Title",img)
    if b==False:
        continue
    faces=face_detector.detectMultiScale(img,1.06,5)
    
    if (len(faces)==0):
        continue
    for f in faces:
        #print(f)
        x,y,w,h=f
        img1=img[y:y+h,x:x+w]
        img1=cv2.resize(img1,(100,100))
        #print(img1.shape)
        name=knn(pics,Y,img1)
        #print(idx2name[name])
        green=(0,255,0)
        cv2.rectangle(img,(x,y),(x+w,y+h),green,5)
        font=cv2.FONT_HERSHEY_SIMPLEX
        fontscale=1
        xx=cv2.putText(img,idx2name[name],(x,y-8),font,fontscale,green,2,cv2.LINE_AA)
        cv2.imshow('img',xx)
    if cv2.waitKey(1)==ord('q'):
        break
    
cam.release()            
cv2.destroyAllWindows()
